# Ground State Energy Estimation for Photosynthesis with Quantum Circuits

An area where quantum chemistry can offer immediate solutions to the ever-growing energy demand and climate change challenges is by enhancing our understanding of artificial photosynthesis. Artificial photosynthesis aims to replicate and optimize the natural process of converting sunlight, water, and carbion dioxide into energy-rich fuels, resulting in a more sustainable and carbon-neutral energy cycle.

Artifical photosynthesis is a multi-step process that begins with the absorption of sunlight, leading to charge separation and the oxidation of water ($H_{2}0$), which produces oxygen ($O_2$), protons ($H^+$), and electrons ($e^-$).

The electrons and protons extracted from the previous step are used for $CO_2$ reduction to faciliate the production of fuels. This involves reducing $CO_2$ into either CO, hydrocarbons like methan, or other carbonhydrates such as glucose. 

Water oxidation reaction:
$$ 2H_20 \rightarrow O_2 + 4(H^+)+4e^- $$

$CO_2$ reduciton reactions:

\begin{gather*}
\text{Reduction I}:   CO_2 + 2(H^+) + 2e^- \rightarrow CO + H_2O \\
\text{Reduction II}:  CO_2 + 6(H^+) + 6e^- \rightarrow CH_3OH + H_2O \\
\text{Reduction III}: CO_2 + 8(H^+) + 8e^- \rightarrow CH_4 +2H_2O
\end{gather*}

In this notebook, we dive into the process of down-selecting a pool of catalysts for water oxidation and catalysts for the $CO_{2} $ reduction reaction. For each mechanism, the ultimate goal is to estimate the ground state electronic structure per transition states to calculate the energy barrier for the reaction pathway to unconver superior catalysts. 

There are five main steps to estimating the ground state energy of the reaction:

1. Generate the electronic hamiltonian in the second-quantization form (also known as molecular hamiltonian) per state of the reaction
   1. #TODO: Write why to use second quantization formulation vs first quantization
2. Prepare an initial state that provides sufficient overlap with the true ground states, boosting the success probability of the phase estimation for the reactive system. This can be achieved by the linear combination of the HF state and selected configuration interaction singles (CIS) states. These multireference states can be easily prepared with rotations and entanglement gates.
3. Perform a mapping between Qubit operators and Fermionic operators. This can be achieved through methods such as Jordan-Wigner, Bravyi-Kitaev, or parity encoding. For this notebook, we will be using Jordan-Wigner, which is handled by pyLIQTR, to perform this mapping.
4. Perform Ground State Energy Estimation on the mapped qubit hamiltonian to estimate the activation energy of the reaction.
5. With the qubit Hamiltonian and initial state, we are now ready to measure the energies of the reaction of interest.

Typical examples of water oxidation:
$CO_4O_4$ catalysis (https://pubs.acs.org/doi/10.1021/ja202320q)

In [1]:
import re
import sys
import time
import json
import numpy as np
from openfermionpyscf import run_pyscf
from openfermion.chem import MolecularData
from pyLIQTR.PhaseEstimation.pe import PhaseEstimation
from qca.utils.utils import extract_number, circuit_estimate

# Hamiltonian Generation
First, we will define the functions necessary to grab the charge, multiplicity, and number of atoms for some molecule within a desired pathway that was specified from a catalyst of interest. Once we have such information, we can then use it to construct a molecular hamiltonian along the reaction pathway of interest. 

Our input for this is a file encoded in the XYZ file format is used for depicting molecular data. An XYZ file gives the number of atoms of the molecule on the first line followed by the molecule's charge, multiplicty and atomic symbol. Recall the following:
- The charge is defined as an integer giving the total molecular charge
- The multiplicity is an integer giving the spin multiplicity, which is the number of probable orientations of the spin angular momentum corresponding to a given total spin quantum number

This is followed by the cartesian coordinates of each atom in the molecule. This notebook comes shipped with an XYZ file that describes water oxidation by using $Co_4O_4$ as a catalyst and $CO_2$ reduction by using $CoPc$ as a catalyst. One can take a look at such files in the data/ directory in this repository.

In [2]:
t_init = time.perf_counter()
def grab_line_info(current_line:str):
    multiplicity = 0
    charge = 0
    multiplicity_match = re.search(r"multiplicity\s*=\s*(\d+)", current_line)
    if multiplicity_match:
        multiplicity = int(multiplicity_match.group(1))
    charge_match = re.search(r"charge\s*=\s*(\d+)", current_line)
    if charge_match:
        charge = int(charge_match.group(1))
    return multiplicity, charge

def grab_pathway_info(data: list[str], nat:int, current_line:str, coord_pathways:list, current_idx:int):
    coords_list = []
    multiplicity, charge = grab_line_info(current_line)
    coords_list.append([nat, charge, multiplicity])
    for point in range(nat):
        data_point = data[current_idx+1+point].split()
        aty = data_point[0]
        xyz = [float(data_point[i]) for i in range(1,4)]
        coords_list.append([aty, xyz])
    coord_pathways.append(coords_list)

In [3]:
def load_pathway(fname:str, pathway:list[int]=None) -> list:
    with open(fname, 'r') as f:
        coordinates_pathway = []
        data = f.readlines()
        data_length = len(data)
        idx = 0
        while idx < data_length:
            line = data[idx]
            if 'charge' in line or 'multiplicity' in line:
                geo_name = ''
                if len(line.split(',')) > 2:
                    geo_name = line.split(',')[2]
                nat = int(data[idx-1].split()[0])
                if geo_name and pathway:
                    order = extract_number(geo_name)
                    if order and order in pathway:
                        grab_pathway_info(data, nat, line, coordinates_pathway, idx)
                else:
                    grab_pathway_info(data, nat, line, coordinates_pathway, idx)
                idx += nat + 2
            else:
                idx += 1
    return coordinates_pathway

We then define the appropriate parameters for generating the electronic hamiltonian along a reaction pathway. The Python-based Simulations of Chemistry Framework (PySCF) is an open-source collection of electronic structure modules and we interface it through an openfermion plugin called openfermionpyscf. openfermionpyscf is actually what is used to generate the molecular hamiltonian.

The calculation parameters are used to indicate whether we want to perform a specific calculation. They are as follows:
- run_scf: boolean flag to indicate running an SCF calculation
- run_mp2: boolean flag to indicate running a MP2 calculation
- run_cisd: boolean flag to indicate running a CISD calculation
- run_ccsd: boolean flag to indicate running a CCSD calculation
- run_fci: boolean flag to indicate running a FCI calculation

Additionally, we need to choose the basis set for our molecular hamiltonian. There are different basis sets we can choose from, but for the purpose of minimizing computational complexity, we choose 'sto-3g' as our basis set as its a common minimal basis set and is the cheapest to compute.

In the selection of an active space, the key principle is that all strongly correlated orbitals must be identified and selected to active space. Given how the active space is selected, we can effectively reduce the size of the molecular hamiltonian and the subsequent qubit hamiltonian.

For ease of use for running the notebook, the pathway provided for water oxidation via a $Co_4O_4$ catalyst results in a simple and minimal solution. There are other pathways that one can choose from, but the pathway specified is sufficient for our use case and has the benefit of fast compilation. 


In [4]:
molecular_hamiltonians = []
pathway = [1, 14, 15, 16, 24, 25, 26, 27]

# water oxidation via Co4O4 catalyst.
coordinates_pathway = load_pathway('../data/water_oxidation_Co4O4.xyz', pathway=pathway)

# Set calculation parameters.
run_scf = 1
run_mp2 = 0
run_cisd = 0
run_ccsd = 0
run_fci = 0

# Set molecule parameters.
basis = 'sto-3g'
active_space_frac = 10

The decision for choosing the initial state in a quantum model pays a pivotal role in achieving accurate results, particularly in capturating quantum systems' true ground state energies. Here, we have the initial state of each molecular structure be constructed based on the hamiltonian.

In the case of preparing the initial state, a Hartree-Fock(HF) computation serves as a good initial approximation. The initial state of each molecular hamiltonian will be its respective Fock state after performing a HF computation on it.

In [6]:
def generate_electronic_hamiltonians(coordinates_pathway:list) -> list:
    molecular_hamiltonians = []
    for idx, coords in enumerate(coordinates_pathway):
        t_coord_start = time.perf_counter()
        _, charge, multi = [int(coords[0][j]) for j in range(3)]

        # set molecular geometry in pyscf format
        geometry = []
        for coord in coords[1:]:
            atom = (coord[0], tuple(coord[1]))
            geometry.append(atom)
        
        molecule = MolecularData(geometry=geometry,
                                 basis=basis,
                                 multiplicity=multi,
                                 charge=charge,
                                 description='catalyst')
        t0 = time.perf_counter()
        molecule = run_pyscf(molecule,
                             run_scf=run_scf,
                             run_mp2=run_mp2,
                             run_cisd=run_cisd,
                             run_ccsd=run_ccsd,
                             run_fci=run_fci)
        t1 = time.perf_counter()

        print(f'Time to perform a HF calculation on molecule {idx} : {t1-t0}')
        print(f'Number of orbitals          : {molecule.n_orbitals}')
        print(f'Number of electrons         : {molecule.n_electrons}')

        print(f'Number of qubits            : {molecule.n_qubits}')
        print(f'Hartree-Fock energy         : {molecule.hf_energy}')
        sys.stdout.flush()

        nocc = molecule.n_electrons // 2
        nvir = molecule.n_orbitals - nocc
        
        print(f'Number of unoccupied Molecular orbitals are: {nvir}')
        print(f'Number of occupied Molecular orbitals are: {nocc}')
        sys.stdout.flush()

        # get molecular Hamiltonian
        active_space_start =  nocc - nocc // active_space_frac # start index of active space
        active_space_stop = nocc + nvir // active_space_frac   # end index of active space

        print(f'active_space start : {active_space_start}')
        print(f'active_space stop  : {active_space_stop}')
        sys.stdout.flush()

        molecular_hamiltonian = molecule.get_molecular_hamiltonian(
            occupied_indices=range(active_space_start),
            active_indices=range(active_space_start, active_space_stop)
        )
        print(f'n qubits are {molecular_hamiltonian.n_qubits}')

        # shifted by HF energy
        molecular_hamiltonian -= molecule.hf_energy
        molecular_hamiltonians.append([molecular_hamiltonian, molecule.hf_energy])
        t_coord_end = time.perf_counter()
        print(f'Time to generate a molecular hamiltonian for molecule {idx} : {t_coord_end-t_coord_start}\n')
    return molecular_hamiltonians


if coordinates_pathway:
    molecular_hamiltonians = generate_electronic_hamiltonians(coordinates_pathway)

Time to perform a HF calculation on molecule 0 : 50.073692375000974
Number of orbitals          : 100
Number of electrons         : 148
Number of qubits            : 200
Hartree-Fock energy         : -3479.3603932694523
Number of unoccupied Molecular orbitals are: 26
Number of occupied Molecular orbitals are: 74
active_space start : 67
active_space stop  : 76
n qubits are 18
Time to generate a molecular hamiltonian for molecule 0 : 50.089522000002034

Time to perform a HF calculation on molecule 1 : 61.49441420899893
Number of orbitals          : 99
Number of electrons         : 147
Number of qubits            : 198
Hartree-Fock energy         : -3478.738701637257
Number of unoccupied Molecular orbitals are: 26
Number of occupied Molecular orbitals are: 73
active_space start : 66
active_space stop  : 75
n qubits are 18
Time to generate a molecular hamiltonian for molecule 1 : 61.523372333002044

Time to perform a HF calculation on molecule 2 : 59.77251950000209
Number of orbitals      

KeyboardInterrupt: 

# Ground State Energy Estimation
Once the initial state is prepared, we can now perform Quantum Phase Estimation(QPE) to estimate the ground state energy of each molecular hamiltonian along the reaction pathway. The initial prepared state used is the Fock state for each molecular hamiltonian. Currently, to simplify the problem of estimating the ground state energy, we are using a short evolution time and a second order trotterization with a single step. Scaling arguments are used to determine the final resources since generating the full circuit for a large number of trotter steps with many bits of precision is quite costly and will increase the compilation time.

Though not shown explicitly here, when we pass the molecular hamiltonian as arguments to generating a circuit for estimating the ground state energy, pyLIQTR performs a Jordan-Wigner transformation on it. This operation maps the fermionic operators to qubit operators, now allowing us to apply quantum algorithms on the hamiltonian. 

Once a circuit is generated to estimate the ground state energy for each molecular hamiltonian, we translate the circuit to a fault tolerant gate set, i.e, clifford + T gateset, to grab its resource estimates. The resource estimates are encoded as JSON files which contains all of the resource estimates for a given molecular hamiltonian.

In [ ]:
# Confirm with yu Zhang and/or andrey
def grab_molecular_phase_offset(hf_energy: float):
    E_min = -0.25 * hf_energy
    E_max = 0
    omega = E_max - E_min
    t = 2*np.pi/omega
    return E_max * t

In [ ]:
trotter_order = 2
trotter_steps = 1
bits_precision = 1

gse_args = {
    'trotterize' : True,
    'ev_time'    : 1,
    'trot_ord'   : trotter_order,
    'trot_num'   : trotter_steps
}
resource_estimates = []

for idx, molecular_hamiltonian_info in enumerate(molecular_hamiltonians):
    molecular_hamiltonian = molecular_hamiltonian_info[0]
    molecular_hf_energy = molecular_hamiltonian_info[1]
    
    n_qubits = molecular_hamiltonian.n_qubits
    gse_args['mol_ham'] = molecular_hamiltonian
    phase_offset = grab_molecular_phase_offset(molecular_hf_energy)
    init_state = [0] * n_qubits #TODO: use Fock state from Hartree-Fock as initial state
    
    t0 = time.perf_counter()
    gse_inst = PhaseEstimation(
        precision_order=bits_precision,
        init_state=init_state,
        phase_offset=phase_offset,
        include_classical_bits=False,
        kwargs=gse_args
    )
    gse_inst.generate_circuit()
    t1 = time.perf_counter()

    print(f'Time to generate a circuit for estimating the GSE for Co4O4 {idx}: {t1-t0}')
    gse_circuit = gse_inst.pe_circuit
    
    t0 = time.perf_counter()
    mh_resources = circuit_estimate(
                        gse_circuit,
                        outdir='GSE/Quantum_Chemistry',
                        circuit_name=f'Co4O4_{idx}',
                        trotter_steps=trotter_steps,
                        write_circuits=True
                    )
    resource_estimates.append(mh_resources)
    t1 = time.perf_counter()
    print(f'Time to estimate Co4O4_step({idx}): {t1-t0}')

Time to generate a circuit for estimating the GSE for Co4O4 0: 0.25770729199575726
   Time to decompose high level <class 'cirq.ops.common_gates.HPowGate circuit: 8.850000449456275e-05 seconds 
   Time to transform decomposed <class 'cirq.ops.common_gates.HPowGate circuit to Clifford+T: 2.3999993572942913e-05 seconds
   Time to decompose high level <class 'cirq.ops.identity.IdentityGate circuit: 3.3208998502232134e-05 seconds 
   Time to transform decomposed <class 'cirq.ops.identity.IdentityGate circuit to Clifford+T: 2.5410117814317346e-06 seconds
   Time to decompose high level <class 'pyLIQTR.PhaseEstimation.pe_gates.PhaseOffset circuit: 0.00013495799794327468 seconds 
   Time to transform decomposed <class 'pyLIQTR.PhaseEstimation.pe_gates.PhaseOffset circuit to Clifford+T: 0.00019212499319110066 seconds
   Time to decompose high level <class 'pyLIQTR.PhaseEstimation.pe_gates.Trotter_Unitary circuit: 41.94738395899185 seconds 
   Time to transform decomposed <class 'pyLIQTR.PhaseE

Now that we've estimated the ground state energy for each step of the reaction pathway, we can calculate the activation energy. The activation energy is essential in assessing the feasibility and kinetics of a reaction and directly influencing the efficiency of a catalyst. By accurately computing the activation energies for each step of the reaction pathway, we can predict the rate at which the reaction will proceed under different conditions.

The activation energy is crucial for determining rate constants. The rate constant(k) is the proportionality constant that expresses the relationship between the rate of a chemical reaction and the concentrations of the reacting substances. Its shown in the Arrhenius equation:
$$k \propto exp(-E_a /RT) $$
where R is the universal gas constant, T is the temperature, and $E_a$ is the activation energy of the reaction. This calculation is essential for determining the quality of a catalyst.

In [ ]:
# TODO: Calculate activation energy along the reaction pathway

In [ ]:
t_end = time.perf_counter()
print(f'Total time to run through this notebook: {t_end-t_init}')

Total time to run through this notebook: 3804.477998625007
